In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Number Partitioning (2 sets)

Number partitioning asks the following question: Given a set of $N$ positive numbers $S = \{ n_{0}, \ldots , n_{N-1}\}$, is there a partition of $S$ into two disjoint subsets $R$ and $S − R$, such that the sum of numbers in both sets is the same? For instance: Can a set of $N$ assets with values $n_0, \ldots , n_{N-1}$ fairly be distributed between two parties? For example a valid solution to partitioning the set $S = \{3,1,1,2,2,1\}$ is $R = \{1,1,1,2\}$ and $S-R = \{2,3\}$.

The Quadratic Unconstrained Binary Optimization model can be formulated in the following way:

$$
H = \alpha \left( \sum_{n=0}^{N-1} n_{i} x_{i} - \frac{1}{2} \sum_{n=0}^{N-1} n_{i} \right)^2 \, .
$$

The decision variable $x_{i} \in \{0,1\}$ is defined as
$$
x_{i} = 
\left\lbrace
\begin{matrix} 1, & \text{if number } n_i \text{ of set } S \text{ is in set } R \\ 0, & \text{else}
\end{matrix}
\right.
$$

If you want no know more about Number Partitioning, you can view the [Wikipedia](https://en.wikipedia.org/wiki/Partition_problem) article or [Lukas2014](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full).

### 1. Step: Define $N$, and create $S$

In [ ]:
import matplotlib.pyplot as plt
import random
from IPython.display import display, HTML
from ipywidgets import interact

data = {'N': None, 'S': None}

def f(N, seed):
    data['N'] = N
    random.seed(seed)
    data['S'] = [random.uniform(0.0, 1.0) for _ in range(N)]

    fig, ax = plt.subplots(1, 1)
    fig.canvas.header_visible = False
    plt.scatter(list(range(N)), data['S'])
    plt.yticks([0.0, 1.0])
    plt.ylabel('Value')
    plt.xticks([0, N - 1])
    plt.xlabel('Item')
    plt.title(f"total  = {sum(data['S']):.4f}")
    plt.show()

display(HTML('<h3>You can modify N and the random seed for the values:</h3>'))
interact(f, N=(0, 200, 4), seed=(0, 84));

### 2. Step: Build $H$ based on the formulation above

In [ ]:
from dadk.BinPol import *

alpha = 1.0

H = alpha * (BinPol.sum(Term(data['S'][i], i) for i in range(data['N'])) - sum(data['S']) / 2) ** 2

print(f"H with {H.N} bits created.")

### 3. Step: A new solver instance is created, and the minimize method is executed

In [ ]:
from dadk.QUBOSolverCPU import *

solver = QUBOSolverCPU(
    number_iterations=H.N**2,            # Total number of iterations per run.
    number_runs=4,                       # Number of stochastically independent runs.
    temperature_start=0.01,              # Start temperature of the annealing process.
    temperature_end=0.0001,              # End temperature of the annealing process.
    temperature_mode=0,                  # 0, 1, or 2 to define the cooling curve:
                                         #    0, 'EXPONENTIAL':
                                         #       reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                         #    1, 'INVERSE':
                                         #       reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                         #    2, 'INVERSE_ROOT':
                                         #       reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_interval=data['N'],      # Number of iterations keeping temperature constant.
    offset_increase_rate=0.00005,        # Increase of dynamic offset when no bit selected. Set to 0.0 to switch off dynamic energy feature.
    bit_precision=64,                    # Bit precision (DAU version 2).
    graphics=GraphicsDetail.ALL,         # Switch on graphics output.
    auto_tuning=AutoTuning.AUTO_SCALING, # Following methods for scaling ``qubo`` and determining temperatures are available:
                                         #    AutoTuning.NOTHING:
                                         #       no action
                                         #    AutoTuning.SCALING:
                                         #       ``scaling_factor`` is multiplied to ``qubo``, ``temperature_start``, ``temperature_end`` and ``offset_increase_rate``.
                                         #    AutoTuning.AUTO_SCALING:
                                         #       A maximum scaling factor w.r.t. ``scaling_bit_precision`` is multiplied to ``qubo``, ``temperature_start``, ``temperature_end`` and ``offset_increase_rate``.
                                         #    AutoTuning.SAMPLING:
                                         #       ``temperature_start``, ``temperature_end`` and ``offset_increase_rate`` are automatically determined.
                                         #    AutoTuning.AUTO_SCALING_AND_SAMPLING:
                                         #       Temperatures and scaling factor are automatically determined and applied.
    scaling_bit_precision=62             # Maximum bit_precision for ``qubo``. Used to define the scaling factor for ``qubo``, ``temperature_start``, ``temperature_end`` and ``offset_increase_rate``.
)

solution_list = solver.minimize(H)
print(KEYWORD_EXECUTION_TIME, solution_list.solver_times.duration_execution)

### 4. Step: Check the results

As you can see in the table, 4 runs are executed (`number_runs=4`) and the result for each run is shown. You can change the parameters and test other settings.

In [ ]:
from tabulate import tabulate

table = []
for solution in solution_list.solutions:
    results = [sum(data['S'][i] for i in range(data['N']) if solution.configuration[i] == (1 - p)) for p in range(2)]
    deltas = [sum(data['S']) / 2 - results[p] for p in range(2)]
    table.append([int(solution.frequency), int(solution.energy)] + results + deltas)

header = ['frequency', 'energy'] + ['sum numbers set R', 'sum numbers set S-R'] + ['delta$_{S}$', 'delta$_{S-R}$']
formats = ['d', 'd'] + 2 * ['.8f'] + 2 * ['.8e']
display(HTML(tabulate(table, header, tablefmt='html', floatfmt=formats)))

### 5. Step: Check the annealing graphics to tune the parameter

In [ ]:
solution_list.display_graphs()